In [1]:
import sys
sys.path.append('../scripts/')
from utility import read_file, create_folder, temp_record_query, temp_record_sdf

In [2]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

In [3]:
spark = (
    # Create a spark session (which will run spark jobs)
    SparkSession.builder.appName("Project 2")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config('spark.executor.memory','10g')
    .config('spark.driver.memory','12g')
    .config('spark.driver.maxResultsSize', '10GiB')
    # .config("spark.network.timeout", "3600s")
    # .master("local[6]")
    .getOrCreate()
    )

22/10/03 11:06:28 WARN Utils: Your hostname, Runyus-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.0.103 instead (on interface en0)
22/10/03 11:06:28 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/03 11:06:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/10/03 11:06:29 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [8]:
trans_sdf = read_file(spark, 'changed_data', '../data/curated/')

|> Loading File...
|> Loading Finished!
-RECORD 0--------------------------------------------------------------------------
 user_id           | 8972                                                          
 merchant_abn      | 59658193201                                                   
 dollar_value      | 182.16129389048888                                            
 order_id          | 5a2112b5-6a59-4ceb-95a5-655fa5915c7a                          
 merchant_name     | Id Ante Institute                                             
 tags              | cable, satellite, and other pay television and radio services 
 take_rate         | 6.16                                                          
 type              | a                                                             
 postcode          | 5311                                                          
 consumer_id       | 1115127                                                       
 name              | Mr. Bradley Moo

In [9]:
consumer_fraud_sdf = read_file(spark,'consumer_fraud_probability.csv', '../data/tables/', 'csv')

|> Loading File...
|> Loading Finished!
-RECORD 0-----------------------------
 user_id           | 6228             
 order_datetime    | 2021-12-19       
 fraud_probability | 97.6298077657765 
only showing top 1 row



In [ ]:
consumer_fraud_sdf = (consumer_fraud_sdf
                   .withColumnRenamed("user_id", 'id')
                   .withColumnRenamed('order_datetime','total_retail')
                   )

In [ ]:
consumer_fraud_table = trans_sdf.join(consumer_fraud_sdf, how='inner', on=(consumer_fraud_sdf.order_datetime == trans_sdf.order) < 1) & (F.months_between(sdf.order_datetime, rts_sdf.month) >= 0))